In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings('ignore') 

In [2]:
def accuracy(actual, predictions):
    logr_accuracy_all_predictors = np.mean(actual == predictions)
    return logr_accuracy_all_predictors  # Corrected indentation

def calibration(actual, predictions):
    return np.sum(predictions) / np.sum(actual)

def logloss(actual, predictions):
    epsilon = 1e-15
    predictions = np.clip(predictions, epsilon, 1 - epsilon)
    
    logr_logloss_all_predictors = -(1 / len(actual)) * np.sum(
        actual * np.log(predictions) + (1 - actual) * np.log(1 - predictions))
    return logr_logloss_all_predictors

In [3]:
def calculate_implied_probabilities(odds_w, odds_l):
    if odds_w == 0 or odds_l == 0 or pd.isna(odds_w) or pd.isna(odds_l):
        return 0, 0 
    implied_prob_w = 1 / odds_w
    implied_prob_l = 1 / odds_l
    overround = implied_prob_w + implied_prob_l
    normalized_prob_w = implied_prob_w / overround
    normalized_prob_l = implied_prob_l / overround
    return normalized_prob_w, normalized_prob_l

In [4]:
def evaluate_predictions(actual_outcomes, binary_predictions,  probability_predictions):
    
    accuracy_result = accuracy(actual_outcomes, binary_predictions)
    
    calibration_result = calibration(actual_outcomes, probability_predictions)
    
    logloss_result = logloss(actual_outcomes, probability_predictions)
    
    return {
        'accuracy': accuracy_result,
        'calibration': calibration_result,
        'log_loss': logloss_result
    }

In [5]:
tennis_dfs = {}

In [6]:
for year in range(2000, 2020): 
    file_extension = 'xls' if year < 2013 else 'xlsx'
    file_path = f"Betting_Odds_Tennis/{year}.{file_extension}"
    tennis_dfs[year] = pd.read_excel(file_path)

In [7]:
tennis_dfs[2019]

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,Lsets,Comment,B365W,B365L,PSW,PSL,MaxW,MaxL,AvgW,AvgL
0,1,Brisbane,Brisbane International,2018-12-31,ATP250,Outdoor,Hard,1st Round,3,Dimitrov G.,...,0.0,Completed,1.36,3.00,1.36,3.37,1.42,3.60,1.35,3.18
1,1,Brisbane,Brisbane International,2018-12-31,ATP250,Outdoor,Hard,1st Round,3,Raonic M.,...,0.0,Completed,1.18,4.50,1.23,4.68,1.27,4.84,1.22,4.26
2,1,Brisbane,Brisbane International,2018-12-31,ATP250,Outdoor,Hard,1st Round,3,Kecmanovic M.,...,0.0,Completed,1.57,2.25,1.67,2.32,1.71,2.40,1.63,2.28
3,1,Brisbane,Brisbane International,2018-12-31,ATP250,Outdoor,Hard,1st Round,3,Millman J.,...,1.0,Completed,1.40,2.75,1.41,3.13,1.45,3.20,1.40,2.95
4,1,Brisbane,Brisbane International,2018-12-31,ATP250,Outdoor,Hard,1st Round,3,Uchiyama Y.,...,0.0,Completed,2.62,1.44,2.73,1.51,3.26,1.53,2.69,1.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2605,66,London,Masters Cup,2019-11-15,Masters Cup,Indoor,Hard,Round Robin,3,Nadal R.,...,1.0,Completed,1.44,2.75,1.39,3.26,1.48,3.30,1.41,2.93
2606,66,London,Masters Cup,2019-11-15,Masters Cup,Indoor,Hard,Round Robin,3,Zverev A.,...,0.0,Completed,1.90,1.90,2.14,1.79,2.24,2.06,1.92,1.90
2607,66,London,Masters Cup,2019-11-16,Masters Cup,Indoor,Hard,Semifinals,3,Tsitsipas S.,...,0.0,Completed,3.50,1.30,3.75,1.33,3.75,1.40,3.39,1.33
2608,66,London,Masters Cup,2019-11-16,Masters Cup,Indoor,Hard,Semifinals,3,Thiem D.,...,0.0,Completed,1.80,2.00,1.84,2.10,1.87,2.20,1.78,2.06


In [8]:
all_years_df = pd.concat(tennis_dfs.values(), ignore_index=True)

In [9]:
all_years_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54908 entries, 0 to 54907
Data columns (total 54 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ATP         54908 non-null  int64         
 1   Location    54908 non-null  object        
 2   Tournament  54908 non-null  object        
 3   Date        54908 non-null  datetime64[ns]
 4   Series      54908 non-null  object        
 5   Court       54908 non-null  object        
 6   Surface     54908 non-null  object        
 7   Round       54908 non-null  object        
 8   Best of     54908 non-null  int64         
 9   Winner      54908 non-null  object        
 10  Loser       54908 non-null  object        
 11  WRank       54889 non-null  object        
 12  LRank       54816 non-null  object        
 13  W1          54624 non-null  float64       
 14  L1          54626 non-null  float64       
 15  W2          54102 non-null  object        
 16  L2          54103 non-

In [10]:
all_years_df.columns

Index(['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface',
       'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'W1', 'L1',
       'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets', 'Lsets',
       'Comment', 'CBW', 'CBL', 'GBW', 'GBL', 'IWW', 'IWL', 'SBW', 'SBL',
       'B365W', 'B365L', 'B&WW', 'B&WL', 'EXW', 'EXL', 'PSW', 'PSL', 'WPts',
       'LPts', 'UBW', 'UBL', 'LBW', 'LBL', 'SJW', 'SJL', 'MaxW', 'MaxL',
       'AvgW', 'AvgL'],
      dtype='object')

In [11]:
def preprocess_dataset(df):

    # Not considering Max and Avg columns

    odds_columns = ['B365W', 'B365L', 'B&WW', 'B&WL', 'CBW', 'CBL', 'EXW', 'EXL', 'LBW', 'LBL',
                'GBW', 'GBL', 'IWW', 'IWL', 'PSW', 'PSL', 'SBW', 'SBL', 'SJW', 'SJL',
                'UBW', 'UBL']


    df['WRank'].replace('NR', np.nan, inplace=True)
    df['LRank'].replace('NR', np.nan, inplace=True)

    df['WRank'] = df['WRank'].fillna(100000).astype(float)
    df['LRank'] = df['LRank'].fillna(100000).astype(float)

    df['higher_rank_won'] = (df['WRank'] < df['LRank']).astype(int)
    df['higher_rank_points'] = df['higher_rank_won'] * df['WPts'] + df['LPts'] * (1 - df['higher_rank_won'])
    df['lower_rank_points'] = (1 - df['higher_rank_won']) * df['WPts'] + df['LPts'] * df['higher_rank_won']
    df['points_diff'] = df['higher_rank_points'] - df['lower_rank_points']

    essential_columns = ['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface', 'Round', 'Best of', 'higher_rank_won', 'higher_rank_points', 'lower_rank_points', 'points_diff', 'WRank', 'LRank', 'Wsets', 'Lsets']

    df_odds = df[essential_columns + odds_columns]


    df_odds[odds_columns] = df_odds[odds_columns].apply(pd.to_numeric, errors='coerce')


    df_odds['EXW'] = df_odds['EXW'].replace('2.,3', 2.3).astype(float)


    df_odds = df_odds.astype({'Location': 'category', 'Tournament': 'category', 'Series': 'category','Court': 'category', 'Surface': 'category', 'Round': 'category' })


    df_odds['higher_rank_won'] = df_odds['WRank'] < df_odds['LRank']

    categorical_features = ['Location', 'Tournament', 'Series', 'Court', 'Surface','Round']
    encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)  # Ensure output is a dense array

    encoded_features = encoder.fit_transform(df_odds[categorical_features])

    df_encoded = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))

    df_odds.reset_index(drop=True, inplace=True)

    df_final = pd.concat([df_odds.drop(columns=categorical_features), df_encoded], axis=1)

    df_final.dropna(axis=1, how='all', inplace=True)

    return df_final

In [12]:
split_time = "2019-01-01"
df_odds_train = all_years_df[all_years_df.Date < split_time]

In [13]:
df_odds_train.tail()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,UBW,UBL,LBW,LBL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
52356,3,Pune,Maharashtra Open,2018-12-31,ATP250,Outdoor,Hard,1st Round,3,Darcis S.,...,NaN,NaN,NaN,NaN,NaN,NaN,2.47,1.65,2.35,1.59
52357,3,Pune,Maharashtra Open,2018-12-31,ATP250,Outdoor,Hard,1st Round,3,Munar J.,...,NaN,NaN,NaN,NaN,NaN,NaN,2.08,1.95,1.94,1.86
52358,3,Pune,Maharashtra Open,2018-12-31,ATP250,Outdoor,Hard,1st Round,3,Donskoy E.,...,NaN,NaN,NaN,NaN,NaN,NaN,1.57,2.65,1.51,2.53
52359,3,Pune,Maharashtra Open,2018-12-31,ATP250,Outdoor,Hard,1st Round,3,Mmoh M.,...,NaN,NaN,NaN,NaN,NaN,NaN,1.83,2.17,1.74,2.09
52360,3,Pune,Maharashtra Open,2018-12-31,ATP250,Outdoor,Hard,1st Round,3,Gulbis E.,...,NaN,NaN,NaN,NaN,NaN,NaN,1.40,3.50,1.35,3.19


In [14]:
df_odds_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52315 entries, 0 to 52360
Data columns (total 54 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ATP         52315 non-null  int64         
 1   Location    52315 non-null  object        
 2   Tournament  52315 non-null  object        
 3   Date        52315 non-null  datetime64[ns]
 4   Series      52315 non-null  object        
 5   Court       52315 non-null  object        
 6   Surface     52315 non-null  object        
 7   Round       52315 non-null  object        
 8   Best of     52315 non-null  int64         
 9   Winner      52315 non-null  object        
 10  Loser       52315 non-null  object        
 11  WRank       52299 non-null  object        
 12  LRank       52236 non-null  object        
 13  W1          52052 non-null  float64       
 14  L1          52054 non-null  float64       
 15  W2          51543 non-null  object        
 16  L2          51544 non-

In [15]:
df_odds_train = preprocess_dataset(df_odds_train) 

In [16]:
df_odds_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52315 entries, 0 to 52314
Columns: 385 entries, ATP to Round_The Final
dtypes: bool(1), datetime64[ns](1), float64(381), int64(2)
memory usage: 153.3 MB


In [17]:
means = df_odds_train.select_dtypes(include=[np.number]).mean()
df_odds_train.fillna(means, inplace=True)

In [18]:
df_odds_validation = all_years_df[all_years_df.Date >= split_time]
df_odds_validation = preprocess_dataset(df_odds_validation) 

In [19]:
df_odds_validation.fillna(means, inplace=True)

In [20]:
df_odds_validation.head()

,ATP,Date,Best of,higher_rank_won,higher_rank_points,lower_rank_points,points_diff,WRank,LRank,Wsets,...,Surface_Grass,Surface_Hard,Round_1st Round,Round_2nd Round,Round_3rd Round,Round_4th Round,Round_Quarterfinals,Round_Round Robin,Round_Semifinals,Round_The Final
0,1,2019-01-01,3,False,974.0,810.0,164.0,63.0,49.0,2.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2019-01-01,3,True,1050.0,875.0,175.0,40.0,57.0,2.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2019-01-01,3,False,206.0,200.0,6.0,240.0,234.0,2.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,2019-01-01,3,True,1125.0,810.0,315.0,35.0,62.0,2.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,2019-01-01,3,False,367.0,200.0,167.0,239.0,146.0,2.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
df_odds_validation.columns

Index(['ATP', 'Date', 'Best of', 'higher_rank_won', 'higher_rank_points',
       'lower_rank_points', 'points_diff', 'WRank', 'LRank', 'Wsets',
       ...
       'Surface_Grass', 'Surface_Hard', 'Round_1st Round', 'Round_2nd Round',
       'Round_3rd Round', 'Round_4th Round', 'Round_Quarterfinals',
       'Round_Round Robin', 'Round_Semifinals', 'Round_The Final'],
      dtype='object', length=162)

## Naive Model

In [22]:
def higher_ranked_wins(tennis_df):

    matches_won_by_higher_ranked = tennis_df.apply(lambda row: row['WRank'] < row['LRank'], axis=1).sum()

    total_matches = len(tennis_df)

    probability_higher_ranked_wins = matches_won_by_higher_ranked / total_matches

    return probability_higher_ranked_wins

### Metrics for Trained Data

In [23]:
prob_higher_rank_win = higher_ranked_wins(df_odds_train)

In [24]:
naive_predictions = np.full_like(df_odds_train["higher_rank_won"], fill_value = 1)
naive_predictions
naive_probability_predictions = np.full_like(df_odds_train["higher_rank_won"], fill_value = prob_higher_rank_win, dtype = 'float64')                              

In [25]:
evaluate_predictions(df_odds_train["higher_rank_won"], naive_predictions, naive_probability_predictions)

{'accuracy': 0.6553760871642932,
 'calibration': 1.0,
 'log_loss': 0.6440549567249229}

### Metrics for Test Data

In [26]:
naive_predictions = np.full_like(df_odds_validation["higher_rank_won"], fill_value = 1)
naive_predictions
naive_probability_predictions = np.full_like(df_odds_validation["higher_rank_won"], fill_value = prob_higher_rank_win, dtype = 'float64')                              

In [27]:
evaluate_predictions(df_odds_validation["higher_rank_won"], naive_predictions, naive_probability_predictions)

{'accuracy': 0.6139606633243347,
 'calibration': 1.0674561520207362,
 'log_loss': 0.670674949633472}

In [28]:
df_odds_validation

,ATP,Date,Best of,higher_rank_won,higher_rank_points,lower_rank_points,points_diff,WRank,LRank,Wsets,...,Surface_Grass,Surface_Hard,Round_1st Round,Round_2nd Round,Round_3rd Round,Round_4th Round,Round_Quarterfinals,Round_Round Robin,Round_Semifinals,Round_The Final
0,1,2019-01-01,3,False,974.0,810.0,164.0,63.0,49.0,2.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2019-01-01,3,True,1050.0,875.0,175.0,40.0,57.0,2.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2019-01-01,3,False,206.0,200.0,6.0,240.0,234.0,2.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,2019-01-01,3,True,1125.0,810.0,315.0,35.0,62.0,2.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,2019-01-01,3,False,367.0,200.0,167.0,239.0,146.0,2.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2588,66,2019-11-15,3,True,9585.0,4000.0,5585.0,1.0,6.0,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2589,66,2019-11-15,3,False,5705.0,2945.0,2760.0,7.0,4.0,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2590,66,2019-11-16,3,False,6190.0,4000.0,2190.0,6.0,3.0,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2591,66,2019-11-16,3,True,5025.0,2945.0,2080.0,5.0,7.0,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Logistic Model

### Metrics for Trained Data

In [29]:
from sklearn.linear_model import LogisticRegression


logr= LogisticRegression(fit_intercept=False)



logr.fit(df_odds_train[['points_diff']], df_odds_train["higher_rank_won"])

LogisticRegression(fit_intercept=False)

In [30]:
tennis_train_predictions_logr = logr.predict(df_odds_train[['points_diff']])
tennis_train_predictions_logr

tennis_train_prediction_prob_logr = logr.predict_proba(df_odds_train[['points_diff']])[:,1]
tennis_train_prediction_prob_logr

array([0.64441307, 0.64441307, 0.64441307, ..., 0.50718153, 0.50471306,
       0.50695713])

In [31]:
evaluate_predictions(df_odds_train["higher_rank_won"], tennis_train_predictions_logr, tennis_train_prediction_prob_logr)

{'accuracy': 0.6553187422345408,
 'calibration': 0.952152089829394,
 'log_loss': 0.6339817929635292}

### Metrics for Test Data

In [32]:
tennis_validation_predictions_logr = logr.predict(df_odds_validation[['points_diff']])
tennis_validation_predictions_logr

tennis_validation_prediction_prob_logr = logr.predict_proba(df_odds_validation[['points_diff']])[:,1]
tennis_validation_prediction_prob_logr

array([0.51839562, 0.51962824, 0.50067331, ..., 0.7277108 , 0.71781775,
       0.61303746])

In [33]:
evaluate_predictions(df_odds_validation['higher_rank_won'], tennis_validation_predictions_logr,  tennis_validation_prediction_prob_logr)

{'accuracy': 0.61473197069032,
 'calibration': 1.000666797620768,
 'log_loss': 0.6518687198708029}

In [34]:
bookmakers = ['B365', 'PS']

## BCM Model

In [35]:
for bookmaker in bookmakers:
    df_odds_validation[f'{bookmaker}_prob_w'], df_odds_validation[f'{bookmaker}_prob_l'] = zip(*df_odds_validation.apply(lambda row: calculate_implied_probabilities(row[f'{bookmaker}W'], row[f'{bookmaker}L']), axis=1))

In [36]:
 df_odds_validation

,ATP,Date,Best of,higher_rank_won,higher_rank_points,lower_rank_points,points_diff,WRank,LRank,Wsets,...,Round_3rd Round,Round_4th Round,Round_Quarterfinals,Round_Round Robin,Round_Semifinals,Round_The Final,B365_prob_w,B365_prob_l,PS_prob_w,PS_prob_l
0,1,2019-01-01,3,False,974.0,810.0,164.0,63.0,49.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.354680,0.645320,0.347319,0.652681
1,1,2019-01-01,3,True,1050.0,875.0,175.0,40.0,57.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.441489,0.558511,0.435443,0.564557
2,1,2019-01-01,3,False,206.0,200.0,6.0,240.0,234.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.732218,0.267782,0.704497,0.295503
3,1,2019-01-01,3,True,1125.0,810.0,315.0,35.0,62.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.662651,0.337349,0.661290,0.338710
4,1,2019-01-01,3,False,367.0,200.0,167.0,239.0,146.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.410995,0.589005,0.428212,0.571788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2588,66,2019-11-15,3,True,9585.0,4000.0,5585.0,1.0,6.0,2.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.656325,0.343675,0.701075,0.298925
2589,66,2019-11-15,3,False,5705.0,2945.0,2760.0,7.0,4.0,2.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.500000,0.500000,0.455471,0.544529
2590,66,2019-11-16,3,False,6190.0,4000.0,2190.0,6.0,3.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.270833,0.729167,0.261811,0.738189
2591,66,2019-11-16,3,True,5025.0,2945.0,2080.0,5.0,7.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.526316,0.473684,0.532995,0.467005


In [37]:
def logit(p):
    if p == 0 or p == 1 or pd.isna(p):
        return np.nan
    return np.log(p / (1 - p))

def inverse_logit(y):
    if pd.isna(y):
        return np.nan
    return np.exp(y) / (1 + np.exp(y))

In [38]:
for bookmaker in bookmakers :
    df_odds_validation[f'{bookmaker}_logit_prob_w'] = df_odds_validation[f'{bookmaker}_prob_w'].apply(logit)

df_odds_validation['consensus_logit_prob_w'] = df_odds_validation[[f'{bookmaker}_logit_prob_w' for bookmaker in bookmakers]].mean(axis=1, skipna=True)

df_odds_validation['consensus_prob_w'] = df_odds_validation['consensus_logit_prob_w'].apply(inverse_logit)

In [39]:
df_odds_validation

,ATP,Date,Best of,higher_rank_won,higher_rank_points,lower_rank_points,points_diff,WRank,LRank,Wsets,...,Round_Semifinals,Round_The Final,B365_prob_w,B365_prob_l,PS_prob_w,PS_prob_l,B365_logit_prob_w,PS_logit_prob_w,consensus_logit_prob_w,consensus_prob_w
0,1,2019-01-01,3,False,974.0,810.0,164.0,63.0,49.0,2.0,...,0.0,0.0,0.354680,0.645320,0.347319,0.652681,-0.598531,-0.630843,-0.614687,0.350991
1,1,2019-01-01,3,True,1050.0,875.0,175.0,40.0,57.0,2.0,...,0.0,0.0,0.441489,0.558511,0.435443,0.564557,-0.235120,-0.259677,-0.247399,0.438464
2,1,2019-01-01,3,False,206.0,200.0,6.0,240.0,234.0,2.0,...,0.0,0.0,0.732218,0.267782,0.704497,0.295503,1.005903,0.868804,0.937353,0.718565
3,1,2019-01-01,3,True,1125.0,810.0,315.0,35.0,62.0,2.0,...,0.0,0.0,0.662651,0.337349,0.661290,0.338710,0.675129,0.669050,0.672089,0.661971
4,1,2019-01-01,3,False,367.0,200.0,167.0,239.0,146.0,2.0,...,0.0,0.0,0.410995,0.589005,0.428212,0.571788,-0.359855,-0.289152,-0.324503,0.419579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2588,66,2019-11-15,3,True,9585.0,4000.0,5585.0,1.0,6.0,2.0,...,0.0,0.0,0.656325,0.343675,0.701075,0.298925,0.646958,0.852423,0.749691,0.679111
2589,66,2019-11-15,3,False,5705.0,2945.0,2760.0,7.0,4.0,2.0,...,0.0,0.0,0.500000,0.500000,0.455471,0.544529,0.000000,-0.178590,-0.089295,0.477691
2590,66,2019-11-16,3,False,6190.0,4000.0,2190.0,6.0,3.0,2.0,...,1.0,0.0,0.270833,0.729167,0.261811,0.738189,-0.990399,-1.036577,-1.013488,0.266298
2591,66,2019-11-16,3,True,5025.0,2945.0,2080.0,5.0,7.0,2.0,...,1.0,0.0,0.526316,0.473684,0.532995,0.467005,0.105361,0.132172,0.118766,0.529657


In [40]:
predictions = df_odds_validation['consensus_prob_w'] > 0.5

In [41]:
evaluate_predictions(df_odds_validation['higher_rank_won'], predictions, df_odds_validation['consensus_prob_w'] )

{'accuracy': 0.786733513305052,
 'calibration': 0.9562547793740179,
 'log_loss': 0.5031910541956598}